In [1]:
from dotenv import load_dotenv, find_dotenv
import os

In [2]:
# Load environment variables from the .env file
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [3]:
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [4]:
system_template = """Use the following pieces of context to answer the users question.
                     If you don't knwo the answer, just say that you don't know, don't try to make up an answer.
                     {context}

                     Begin!

                     -------------------
                     Question: {question}
                     Answer: """

In [5]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [6]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Robinson Advisory.docx.pdf")
data = loader.load()

In [7]:
'''from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(data)'''

'from langchain.text_splitter import RecursiveCharacterTextSplitter\n\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)\ndocs = text_splitter.split_documents(data)'

In [8]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_text(data[1].page_content)
docs_one = [Document(page_content=t) for t in texts[:]]

In [9]:
print(docs_one)

[Document(page_content='-\n2\n-\n4.\nTerm\n:\nThe\nterm\nof\nthis\nAgreement\nshall\ncommence\non\nthe\nEffective\nDate\nand\nshall\ncontinue\nuntil \nterminated\nin\naccordance\nwith\nthe\nprovisions\nherein\n(the\n"\nTerm\n").\n5.\nTermination\n:\nEither\nparty ,\nat\nany\ngiven\ntime,\nmay\nterminate\nthis\nAgreement,\nfor\nany\nreason \nwhatsoever ,\nwith\nor\nwithout\ncause,\nupon\nfourteen\n(14)\ndays’\nprior\nwritten\nnotice.\nNotwithstanding\nthe \nabove,\nthe\nCompany\nmay\nterminate\nthis\nAgreement\nimmediately\nand\nwithout\nprior\nnotice\nif\nAdvisor \nrefuses\nor\nis\nunable\nto\nperform\nthe\nServices,\nor\nis\nin\nbreach\nof\nany\nprovision\nof\nthis\nAgreement.\n6.\nCompensation:\n6.1\nAs\nfull\nand\nsole\nconsideration\nfor\nthe\nServices,\nand\nsubject\nto\nthe\nperformance\nof\nthe\nServices, \nthe\nCompany\nshall\npay\nthe\nAdvisor ,\nagainst\nan\ninvoice\nprovided\nto\nthe\nCompany\nby\nAdvisor , \nhourly\nfees\nat\na\nrate\nof\nUSD\n9\n(nine)\nper\nBillable\nHour

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents=docs_one,embedding=embedding)

/home/ablaze/miniconda3/envs/week_11_backend/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",openai_api_key = OPENAI_API_KEY)

/home/ablaze/miniconda3/envs/week_11_backend/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
question = "What is ADVISORY SERVICES AGREEMENT"
'''docs = vectorstore.similarity_search(question)
len(docs)'''

'docs = vectorstore.similarity_search(question)\nlen(docs)'

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs = {"prompt": prompt})
result = qa_chain({"query": question})

/home/ablaze/miniconda3/envs/week_11_backend/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [14]:
print(result)

{'query': 'What is ADVISORY SERVICES AGREEMENT', 'result': 'An Advisory Services Agreement is a contract between a company and an advisor or consultant who provides professional advice and expertise to the company. The agreement outlines the scope of services to be provided, the compensation for the advisor, and any other terms and conditions related to the advisory relationship. It also typically includes provisions regarding confidentiality, non-competition, and intellectual property ownership.'}
